# Imports

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'

In [8]:
# Save this cell directly as user_manager.py
#last version
'''%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/user_manager.py"

import pandas as pd
import os

class UserManager:
    def __init__(self, users_csv):
        self.users_csv = users_csv
        if not os.path.exists(self.users_csv):
            pd.DataFrame(columns=['user_id', 'first_name', 'last_name', 'age', 'gender']).to_csv(self.users_csv, index=False)

    def load_users(self):
        return pd.read_csv(self.users_csv)

    def ensure_user(self, user_id, first_name='Unknown', last_name='Unknown', age=0, gender='U'):
        users_df = self.load_users()
        if user_id not in users_df['user_id'].values:
            new_user = pd.DataFrame([{
                'user_id': user_id,
                'first_name': first_name,
                'last_name': last_name,
                'age': age,
                'gender': gender
            }])
            users_df = pd.concat([users_df, new_user], ignore_index=True)
            users_df.to_csv(self.users_csv, index=False)
            print(f"✅ Added new user {user_id}: {first_name} {last_name}")
        else:
            print(f"✅ User {user_id} found: {first_name} {last_name}")

    def get_user(self, user_id):
        users_df = self.load_users()
        user = users_df[users_df['user_id'] == user_id]
        return user.iloc[0].to_dict() if not user.empty else None
'''

'%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/user_manager.py"\n\nimport pandas as pd\nimport os\n\nclass UserManager:\n    def __init__(self, users_csv):\n        self.users_csv = users_csv\n        if not os.path.exists(self.users_csv):\n            pd.DataFrame(columns=[\'user_id\', \'first_name\', \'last_name\', \'age\', \'gender\']).to_csv(self.users_csv, index=False)\n\n    def load_users(self):\n        return pd.read_csv(self.users_csv)\n\n    def ensure_user(self, user_id, first_name=\'Unknown\', last_name=\'Unknown\', age=0, gender=\'U\'):\n        users_df = self.load_users()\n        if user_id not in users_df[\'user_id\'].values:\n            new_user = pd.DataFrame([{\n                \'user_id\': user_id,\n                \'first_name\': first_name,\n                \'last_name\': last_name,\n                \'age\': age,\n                \'gender\': gender\n            }])\n            users_df = pd.concat([users_df, new_user], ignore_index

In [9]:
'''# Save this cell directly as user_manager.py
#%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/user_manager.py"

import pandas as pd
import os
import importlib
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import artist_similarity_service
importlib.reload(artist_similarity_service)
from artist_similarity_service import ArtistSimilarityService

class UserManager:
    def __init__(self, users_csv):
        self.users_csv = users_csv
        if not os.path.exists(self.users_csv):
            pd.DataFrame(columns=['user_id', 'first_name', 'last_name', 'age', 'gender', 'favorite_artists', 'recommended_artists']).to_csv(self.users_csv, index=False)

    def load_users(self):
        return pd.read_csv(self.users_csv)

    def ensure_user(self, user_id, first_name='Unknown', last_name='Unknown', age=0, gender='U'):
        users_df = self.load_users()
        if user_id not in users_df['user_id'].values:
            new_user = pd.DataFrame([{
                'user_id': user_id,
                'first_name': first_name,
                'last_name': last_name,
                'age': age,
                'gender': gender,
                'favorite_artists': '',
                'recommended_artists': ''
            }])
            users_df = pd.concat([users_df, new_user], ignore_index=True)
            users_df.to_csv(self.users_csv, index=False)
            print(f"✅ Added new user {user_id}: {first_name} {last_name}")
        else:
            print(f"✅ User {user_id} found: {first_name} {last_name}")

    def get_user(self, user_id):
        users_df = self.load_users()
        user = users_df[users_df['user_id'] == user_id]
        return user.iloc[0].to_dict() if not user.empty else None

    def get_recommended_artists(self, user_id):
        users_df = self.load_users()
        row = users_df[users_df['user_id'] == user_id]
        if row.empty:
            return []
        val = row.iloc[0].get('recommended_artists', '')
        return [a.strip().lower() for a in str(val).split(',') if a.strip()]

    def update_recommended_artists(self, user_id, artist_list):
        df = self.load_users()
        # Call Deezer-based recommender
        recommender = ArtistSimilarityService()
        similar_artists = recommender.recommend_from_favorites(artist_list)
        recommended_names = [name for name, score in similar_artists if name not in [a.lower() for a in artist_list]]

        df.loc[df['user_id'] == user_id, 'recommended_artists'] = ', '.join(sorted(set(recommended_names)))
        df.to_csv(self.users_csv, index=False)
        print(f"✅ Updated recommended artists for user {user_id}")'''

In [14]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/user_manager.py"

import pandas as pd
import os
import importlib
import sys

# Ensure the custom module path is in sys.path
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')

# Dynamically reload artist similarity service
import artist_similarity_service
importlib.reload(artist_similarity_service)
from artist_similarity_service import ArtistSimilarityService

class UserManager:
    def __init__(self, users_csv):
        self.users_csv = users_csv
        if not os.path.exists(self.users_csv):
            pd.DataFrame(columns=[
                'user_id', 'username', 'password',
                'first_name', 'last_name', 'age', 'gender',
                'favorite_artists', 'recommended_artists'
            ]).to_csv(self.users_csv, index=False)

    def load_users(self):
        return pd.read_csv(self.users_csv)

    def ensure_user(self, user_id, first_name='Unknown', last_name='Unknown', age=0, gender='U'):
        users_df = self.load_users()
        if user_id not in users_df['user_id'].values:
            new_user = pd.DataFrame([{
                'user_id': user_id,
                'first_name': first_name,
                'last_name': last_name,
                'age': age,
                'gender': gender,
                'favorite_artists': '',
                'recommended_artists': ''
            }])
            users_df = pd.concat([users_df, new_user], ignore_index=True)
            users_df.to_csv(self.users_csv, index=False)
            print(f"✅ Added new user {user_id}: {first_name} {last_name}")
        else:
            print(f"✅ User {user_id} found: {first_name} {last_name}")

    def get_user(self, user_id):
        users_df = self.load_users()
        user = users_df[users_df['user_id'] == user_id]
        return user.iloc[0].to_dict() if not user.empty else None

    def get_recommended_artists(self, user_id):
        users_df = self.load_users()
        row = users_df[users_df['user_id'] == user_id]
        if row.empty:
            return []
        val = row.iloc[0].get('recommended_artists', '')
        return [a.strip().lower() for a in str(val).split(',') if a.strip()]

    def update_recommended_artists(self, user_id, artist_list):
        df = self.load_users()

        if not artist_list:
            df.loc[df['user_id'] == user_id, 'recommended_artists'] = ''
            df.to_csv(self.users_csv, index=False)
            print(f"⚠️ Empty favorites: cleared recommended artists for user {user_id}")
            return

        recommender = ArtistSimilarityService()
        try:
            similar_artists = recommender.recommend_from_favorites(artist_list)
            recommended_names = [
                name for name, score in similar_artists
                if name not in [a.lower() for a in artist_list]
            ]
        except Exception as e:
            print(f"❌ Failed to fetch from Deezer: {e}")
            recommended_names = []

        df.loc[df['user_id'] == user_id, 'recommended_artists'] = ', '.join(sorted(set(recommended_names)))
        df.to_csv(self.users_csv, index=False)
        print(f"✅ Updated recommended artists for user {user_id}")


Overwriting /content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/user_manager.py


In [16]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/user_manager.py"

import pandas as pd
import os
import importlib
import sys

# Ensure the custom module path is in sys.path
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')

# Dynamically reload artist similarity service
import artist_similarity_service
importlib.reload(artist_similarity_service)
from artist_similarity_service import ArtistSimilarityService

class UserManager:
    def __init__(self, users_csv):
        self.users_csv = users_csv
        if not os.path.exists(self.users_csv):
            pd.DataFrame(columns=[
                'user_id', 'username', 'password',
                'first_name', 'last_name', 'age', 'gender',
                'favorite_artists', 'recommended_artists'
            ]).to_csv(self.users_csv, index=False)

    def load_users(self):
        return pd.read_csv(self.users_csv)

    def ensure_user(self, user_id, first_name='Unknown', last_name='Unknown', age=0, gender='U'):
        users_df = self.load_users()
        if user_id not in users_df['user_id'].values:
            new_user = pd.DataFrame([{
                'user_id': user_id,
                'first_name': first_name,
                'last_name': last_name,
                'age': age,
                'gender': gender,
                'favorite_artists': '',
                'recommended_artists': ''
            }])
            users_df = pd.concat([users_df, new_user], ignore_index=True)
            users_df.to_csv(self.users_csv, index=False)
            print(f"✅ Added new user {user_id}: {first_name} {last_name}")
        else:
            print(f"✅ User {user_id} found: {first_name} {last_name}")

    def get_user(self, user_id):
        users_df = self.load_users()
        user = users_df[users_df['user_id'] == user_id]
        return user.iloc[0].to_dict() if not user.empty else None

    def get_recommended_artists(self, user_id):
        users_df = self.load_users()
        row = users_df[users_df['user_id'] == user_id]
        if row.empty:
            return []
        val = row.iloc[0].get('recommended_artists', '')
        return [a.strip().lower() for a in str(val).split(',') if a.strip()]



    def update_recommended_artists(self, user_id, artist_list):
      df = self.load_users()
      if not artist_list:
        df.loc[df['user_id'] == user_id, 'recommended_artists'] = ''
        df.to_csv(self.users_csv, index=False)
        print(f"⚠️ Empty favorites: cleared recommended artists for user {user_id}")
        return
      recommender = ArtistSimilarityService()
      try:
        similar_artists = recommender.recommend_from_favorites(artist_list)
        # ✅ Include all, including favorites
        recommended_names = [name for name, score in similar_artists]
      except Exception as e:
        print(f"❌ Failed to fetch from Deezer: {e}")
        recommended_names = []
      df.loc[df['user_id'] == user_id, 'recommended_artists'] = ', '.join(sorted(set(recommended_names)))
      df.to_csv(self.users_csv, index=False)
      print(f"✅ Updated recommended artists for user {user_id}")



Overwriting /content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/user_manager.py
